# history

In [14]:
import os,glob
import time
import copy
import math
import random
import csv
import imageio
import pandas as pd
import numpy as np
from numpy import linalg as LA
import scipy
from scipy.spatial import distance

import SimpleITK as sitk

from matplotlib import pyplot as plt
import seaborn as sns
# import plotly.express as px
# import plotly.graph_objects as go
# from plotly.colors import n_colors

from swc_handler import parse_swc,write_swc,trim_swc,shift_swc,NEURITE_TYPES
from morph_topo import morphology
from file_io import load_image

In [3]:
def write_swcobj_fromtrees(obj_des,trees):
    obj = []
    for tree in trees:
        df = pd.DataFrame(tree,columns=['n','t','x','y','z','r','p'])

        f = []
        k = 0
        id2line = {}
        for i in df.index.tolist():
            idx =  df.loc[i,'n']
            k = k+1
            id2line[idx] = str(k)
            s = ['v' ,str(df.loc[i,'x']),str(df.loc[i,'y']),str(df.loc[i,'z'])]
            f.append(s)
        for i in df.index.tolist():
            if df.loc[i, 'p'] not in id2line.keys():
                s = ['p', id2line[df.loc[i,'n']]]
                f.append(s)
                continue
            s = ['l', id2line[df.loc[i,'n']], id2line[df.loc[i, 'p']]]
            f.append(s)

        obj.append(f)

    length = 0
    new_v = []
    new_p = []
    new_l = []
    for f in obj:
        new_length = 0
        for leaf in f:
            if leaf[0]=='v': 
                new_v.append(leaf)
                new_length += 1
            elif leaf[0]=='p': 
                new_p.append(['p',float(leaf[1])+length])
            elif leaf[0]=='l': 
                new_l.append(['l',float(leaf[1])+length,float(leaf[2])+length])
        length += new_length

    with open(obj_des, 'w+') as fp:
        for i,leaf in enumerate(new_v):
            s = f'{leaf[0]} {leaf[1]} {leaf[2]} {leaf[3]}\n'
            fp.writelines(s)
        for i,leaf in enumerate(new_p):
            s = f'{leaf[0]} {leaf[1]}\n'
            fp.writelines(s)  
        for i,leaf in enumerate(new_l):
            s = f'{leaf[0]} {leaf[1]} {leaf[2]}\n'
            fp.writelines(s) 
            
    return 0

In [6]:
ptype_dir = '../../fig1dptype'
ptype_list = ['CP_SNr/nan/CP_SNr',
              'CP_others/nan/CP_others',
              'CP_GPe/nan/CP_GPe',
              ]
ptype_treedir = '../swctree'
ptype_objdir = '../swcobj'
for ptype in ptype_list:
    print(ptype,)
    ptype_swcfile_list = glob.glob( ptype_dir + '/' + ptype + '*.swc' )   
    for ptype_swcfile in ptype_swcfile_list:
        swcfile = ptype_swcfile.replace(ptype_dir,ptype_treedir)
        os.makedirs(os.path.dirname(swcfile),exist_ok=True)
        print(swcfile)
        cp_str = f'cp {ptype_swcfile} {swcfile}'
        os.system(cp_str)
        
        swctree = parse_swc(swcfile)
        objfile = ptype_swcfile.replace(ptype_dir,ptype_objdir).replace('.swc','.obj')
        os.makedirs(os.path.dirname(objfile),exist_ok=True)
        print(objfile)
        write_swcobj_fromtrees(objfile, [swctree])
    print('\n')
        

CP_SNr/nan/CP_SNr
['../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00001.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00002.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00003.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00005.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00006.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00007.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00009.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00010.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00011.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00027.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00031.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00086.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00093.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00094.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17302_00095.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17545_00006.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17545_00008.swc', '../../fig1dptype/CP_SNr/nan/CP_SNr_17545_00045.swc', '../../fi

swctree/CP_SNr/nan/CP_SNr_17787_00050.swc
swcobj/CP_SNr/nan/CP_SNr_17787_00050.obj
swctree/CP_SNr/nan/CP_SNr_18454_00002.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00002.obj
swctree/CP_SNr/nan/CP_SNr_18454_00003.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00003.obj
swctree/CP_SNr/nan/CP_SNr_18454_00009.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00009.obj
swctree/CP_SNr/nan/CP_SNr_18454_00014.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00014.obj
swctree/CP_SNr/nan/CP_SNr_18454_00018.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00018.obj
swctree/CP_SNr/nan/CP_SNr_18454_00021.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00021.obj
swctree/CP_SNr/nan/CP_SNr_18454_00024.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00024.obj
swctree/CP_SNr/nan/CP_SNr_18454_00028.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00028.obj
swctree/CP_SNr/nan/CP_SNr_18454_00093.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00093.obj
swctree/CP_SNr/nan/CP_SNr_18454_00111.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00111.obj
swctree/CP_SNr/nan/CP_SNr_18454_00174.swc
swcobj/CP_SNr/nan/CP_SNr_18454_00174.obj
swct

swctree/CP_others/nan/CP_others_17302_00055.swc
swcobj/CP_others/nan/CP_others_17302_00055.obj
swctree/CP_others/nan/CP_others_17302_00075.swc
swcobj/CP_others/nan/CP_others_17302_00075.obj
swctree/CP_others/nan/CP_others_17302_00081.swc
swcobj/CP_others/nan/CP_others_17302_00081.obj
swctree/CP_others/nan/CP_others_17302_00087.swc
swcobj/CP_others/nan/CP_others_17302_00087.obj
swctree/CP_others/nan/CP_others_17302_00102.swc
swcobj/CP_others/nan/CP_others_17302_00102.obj
swctree/CP_others/nan/CP_others_17302_00108.swc
swcobj/CP_others/nan/CP_others_17302_00108.obj
swctree/CP_others/nan/CP_others_17545_00001.swc
swcobj/CP_others/nan/CP_others_17545_00001.obj
swctree/CP_others/nan/CP_others_17545_00041.swc
swcobj/CP_others/nan/CP_others_17545_00041.obj
swctree/CP_others/nan/CP_others_17545_00047.swc
swcobj/CP_others/nan/CP_others_17545_00047.obj
swctree/CP_others/nan/CP_others_17545_00147.swc
swcobj/CP_others/nan/CP_others_17545_00147.obj
swctree/CP_others/nan/CP_others_17545_00149.swc
sw

swcobj/CP_GPe/nan/CP_GPe_17302_00016.obj
swctree/CP_GPe/nan/CP_GPe_17302_00017.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00017.obj
swctree/CP_GPe/nan/CP_GPe_17302_00018.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00018.obj
swctree/CP_GPe/nan/CP_GPe_17302_00030.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00030.obj
swctree/CP_GPe/nan/CP_GPe_17302_00033.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00033.obj
swctree/CP_GPe/nan/CP_GPe_17302_00043.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00043.obj
swctree/CP_GPe/nan/CP_GPe_17302_00044.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00044.obj
swctree/CP_GPe/nan/CP_GPe_17302_00045.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00045.obj
swctree/CP_GPe/nan/CP_GPe_17302_00046.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00046.obj
swctree/CP_GPe/nan/CP_GPe_17302_00050.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00050.obj
swctree/CP_GPe/nan/CP_GPe_17302_00056.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00056.obj
swctree/CP_GPe/nan/CP_GPe_17302_00057.swc
swcobj/CP_GPe/nan/CP_GPe_17302_00057.obj
swctree/CP_GPe/nan/CP_GPe_17302_00059.swc
swco

swctree/CP_GPe/nan/CP_GPe_18455_00014.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00014.obj
swctree/CP_GPe/nan/CP_GPe_18455_00026.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00026.obj
swctree/CP_GPe/nan/CP_GPe_18455_00030.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00030.obj
swctree/CP_GPe/nan/CP_GPe_18455_00031.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00031.obj
swctree/CP_GPe/nan/CP_GPe_18455_00032.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00032.obj
swctree/CP_GPe/nan/CP_GPe_18455_00036.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00036.obj
swctree/CP_GPe/nan/CP_GPe_18455_00040.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00040.obj
swctree/CP_GPe/nan/CP_GPe_18455_00041.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00041.obj
swctree/CP_GPe/nan/CP_GPe_18455_00119.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00119.obj
swctree/CP_GPe/nan/CP_GPe_18455_00135.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00135.obj
swctree/CP_GPe/nan/CP_GPe_18455_00172.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00172.obj
swctree/CP_GPe/nan/CP_GPe_18455_00224.swc
swcobj/CP_GPe/nan/CP_GPe_18455_00224.obj
swct